In [1]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
from IPython.display import Audio
from scipy.io import wavfile
import numpy as np
import soundfile
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Model,Wav2Vec2Config, Wav2Vec2FeatureExtractor

In [3]:
import logging
def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger

In [4]:
# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy


In [5]:
from torchmetrics import F1
from torchmetrics import Accuracy

In [6]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class LCNN(nn.Module):

    def __init__(self,mode):

        super(LCNN, self).__init__()
        '''input : (B,1,F,T)'''
        
        self.mode = mode
        
        self.conv2d_1  = nn.Conv2d(1, 32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.conv2d_2  = nn.Conv2d(1, 32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        
        self.max2d_1 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.conv2d_3  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_4  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_1 = nn.BatchNorm2d(32)
        
        self.conv2d_5  = nn.Conv2d(32, 48, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_6  = nn.Conv2d(32, 48, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.max2d_2 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        self.bn2d_2 = nn.BatchNorm2d(48)
        
        
        self.conv2d_7  = nn.Conv2d(48, 48, kernel_size=(1,1), stride=(1,1))
        self.conv2d_8  = nn.Conv2d(48, 48, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_3 = nn.BatchNorm2d(48)
        
        self.conv2d_9  = nn.Conv2d(48, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_10  = nn.Conv2d(48, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.max2d_3 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.conv2d_11  = nn.Conv2d(64, 64, kernel_size=(1,1), stride=(1,1))
        self.conv2d_12  = nn.Conv2d(64, 64, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_4 = nn.BatchNorm2d(64)
        
        self.conv2d_13  = nn.Conv2d(64, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_14  = nn.Conv2d(64, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.bn2d_5 = nn.BatchNorm2d(32)
        
        
        self.conv2d_15  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_16  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_6 = nn.BatchNorm2d(32)
        
        
        self.conv2d_17  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_18  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.max2d_4 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.adaptavg = nn.AdaptiveAvgPool2d(1)
        
        
#        self.fc1 = nn.Linear(64, 128)
        
        if self.mode == 'murmur' :
            self.fc_mur = nn.Sequential(
                                        nn.Linear(32,3),
#                                         nn.Sigmoid()
                                        )
        elif self.mode == 'outcome' :
            self.fc_out = nn.Sequential(
                                        nn.Linear(128,32),
                                        nn.ReLU(),
                                        nn.Linear(32,2)
                                        )
        
        
    def forward(self, input):
        
        x1 = self.conv2d_1(input)
        x2 = self.conv2d_2(input)
        max1 = torch.maximum(x1,x2)
        x = self.max2d_1(max1)
        
        x3 = self.conv2d_3(x)
        x4 = self.conv2d_4(x)
        max2 = torch.maximum(x3,x4)
        x = self.bn2d_1(max2)
        
        x5 = self.conv2d_5(x)
        x6 = self.conv2d_6(x)
        max3 = torch.maximum(x5,x6)
        x = self.max2d_2(max3)
        x = self.bn2d_2(x)
        
        x7 = self.conv2d_7(x)
        x8 = self.conv2d_8(x)
        max4 = torch.maximum(x7,x8)
        x = self.bn2d_3(max4)
        
        x9 = self.conv2d_9(x)
        x10 = self.conv2d_10(x)
        max5 = torch.maximum(x9,x10)
        x = self.max2d_3(max5)
        
        x11 = self.conv2d_11(x)
        x12 = self.conv2d_12(x)
        max6 = torch.maximum(x11,x12)
        x = self.bn2d_4(max6)
        
        x13 = self.conv2d_13(x)
        x14 = self.conv2d_14(x)
        max7 = torch.maximum(x13,x14)
        x = self.bn2d_5(max7)
        
        x15 = self.conv2d_15(x)
        x16 = self.conv2d_16(x)
        max8 = torch.maximum(x15,x16)
        x = self.bn2d_6(max8)
        
        x17 = self.conv2d_17(x)
        x18 = self.conv2d_18(x)
        max9 = torch.maximum(x17,x18)
        x = self.max2d_4(max9)
        
        x = self.adaptavg(x)
        x = x.view(x.size(0),-1)
        
#        x = self.fc1(x)
        
        
        if self.mode == 'murmur' :
            out = self.fc_mur(x)
        elif self.mode == 'outcome' :
            out = self.fc_out(x)
#        out = {'murmur':out1, 'outcome':out2}
        
        return out

In [7]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument('--train_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/',     help='train feature path')
parser.add_argument('--valid_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/',     help='validation feature path')
parser.add_argument('--output_folder',    default='./output_folder_LCNN',     help='output folder')
parser.add_argument("--mode",default="murmur", type=str, help="mode : murmur/outcome")
parser.add_argument("--lr",default=1e-3, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=32, type=int)
parser.add_argument("--minibatchsize_valid", default=1, type=int)
parser.add_argument("--train_num_workers", default=8, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_LCNN', type=str)
parser.add_argument("--logdir", default='./log_hmd_LCNN/', type=str)
parser.add_argument("--model_name", default='hmd_LCNN_model_murmur', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=80, type=int)
parser.add_argument("--gpu", default="0", type=str)




warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [46]:
wav_files = glob.glob(args.train_data_folder + '*.wav')

In [51]:
wav_files[0].split('/')[-1].replace('wav','pk')

'84780_AV.pk'

In [86]:
with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/'+ '84780_AV.pickle' ,'rb') as fr:
    feature = pickle.load(fr)

In [88]:
feature.requires_grad = False

In [89]:
feature

tensor([[[-0.0459,  0.0593, -0.0603,  ..., -0.1019,  0.0114,  0.0242],
         [-0.0499,  0.0556, -0.0516,  ..., -0.0988,  0.0102,  0.0215],
         [-0.0484,  0.0601, -0.0551,  ..., -0.1023,  0.0086,  0.0161],
         ...,
         [-0.0464,  0.0588, -0.0586,  ..., -0.0926,  0.0141,  0.0075],
         [-0.0507,  0.0554, -0.0463,  ..., -0.0952,  0.0089,  0.0145],
         [-0.0652,  0.0647, -0.0358,  ..., -0.1260,  0.0051,  0.0213]]])

In [85]:
from tqdm import tqdm
import pickle
wav_files = glob.glob(args.train_data_folder + '*.wav')
# wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
for i in tqdm(range(len(wav_files))):
    current_recording = read_recording(wav_files[i])
    current_recording = current_recording.reshape(1,80000)
    feature = wav2vec2(torch.tensor(current_recording, dtype=torch.float32))
    feature.requires_grad = False
    fnm = wav_files[i].split('/')[-1].replace('wav','pickle')
    with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/'+ fnm ,'wb') as fw:
        pickle.dump(feature[0],fw)

100%|███████████████████████████████████████| 2532/2532 [06:41<00:00,  6.31it/s]


In [90]:
from tqdm import tqdm
import pickle
wav_files = glob.glob(args.valid_data_folder + '*.wav')
# wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
for i in tqdm(range(len(wav_files))):
    current_recording = read_recording(wav_files[i])
    current_recording = current_recording.reshape(1,80000)
    feature = wav2vec2(torch.tensor(current_recording, dtype=torch.float32))
    feature.requires_grad = False
    fnm = wav_files[i].split('/')[-1].replace('wav','pickle')
    with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/'+ fnm ,'wb') as fw:
        pickle.dump(feature[0],fw)

100%|█████████████████████████████████████████| 631/631 [01:38<00:00,  6.39it/s]


In [72]:
wav_files = glob.glob(args.train_data_folder + '*.wav')
len(wav_files)

2532

In [8]:
from torch.utils.data import Dataset,DataLoader, RandomSampler
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
import librosa
import glob
import math
import pickle





def read_recording(wav_fn, audio_length= 20, sr= 4000):

    recording, frequency = load_wav_file(wav_fn)


    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording



class train_loader(Dataset):
    def __init__(self,data_folder, feature_folder, **kwargs):
        self.data_folder = data_folder
        self.feature_folder = feature_folder
        
        
        self.feature_file = glob.glob(feature_folder + '*.pickle')
        self.wav_files = glob.glob(data_folder + '*.wav')
        self.num_feature_files = len(self.feature_file)
        
        self.murmur_classes = ['Present', 'Absent']
        self.num_murmur_classes = len(self.murmur_classes)
        self.outcome_classes = ['Abnormal', 'Normal']
        self.num_outcome_classes = len(self.outcome_classes)
        
        self.age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
        self.recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
        
        
        

        
        
        examples = {}
        for filename in self.feature_file:
            current_patient_id = filename.split('/')[-1].split('_')[0]
            current_txt_file = self.data_folder + str(current_patient_id) + '.txt'
            with open(current_txt_file, 'r') as f:
                current_patient_data = f.read()
            current_loc = filename.split('/')[-1].split('.')[0].split('_')[1]
            if filename.split('/')[-1].split('.')[0]  not in examples.keys():
                current_feature_data = filename.split('/')[-1].split('.')[0]
                with open(filename, 'rb') as fr:
                    feature = pickle.load(fr)
                examples[current_feature_data] = {
                    "id" : current_patient_id,
                    "txt_fn" : current_txt_file,   #txt,
                    "feature" : feature,  #wav
                    "num_location": get_num_locations(current_patient_data),
                    "frequency" : get_frequency(current_patient_data),
                    "location" : current_loc,
                    "age" : get_age(current_patient_data),
                    "sex" : get_sex(current_patient_data),
                    "height" : get_height(current_patient_data),
                    "weight" : get_weight(current_patient_data),
                    "pregnancy_status" : get_pregnancy_status(current_patient_data),
                    "murmur" : get_murmur(current_patient_data),
                    "outcome" : get_outcome(current_patient_data)

                }
        self.examples = examples
        self.examples_list = list(examples.keys())
        
        
        if len(self.examples_list) != self.num_feature_files:
            raise ValueError('length is different!')
    def __len__(self):
        return len(self.examples_list)
    
        
    def __getitem__(self, index):
        
        # patient info
        current_feature_dict = self.examples[self.examples_list[index]]
        current_patient_data = load_patient_data(current_feature_dict["txt_fn"])

        
        # wav2vec2 feature
        
        feature = current_feature_dict["feature"]
        feature.requires_grad = False
        
        
#         #log mel feature
#         log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = current_recording.astype(np.float32),
#                                                          sr= 4000,
#                                                          n_mels=128,
#                                                          n_fft=400, 
#                                                          hop_length=128, 
#                                                          win_length=400))
        
#         # age
#         current_patient_age = current_wav_dict["age"]
#         current_age_group = np.zeros(6, dtype=np.float32)
#         if current_patient_age in self.age_classes:
#             j = self.age_classes.index(current_patient_age)
#             current_age_group[j] = 1.0
#         else :
#             current_age_group[5] = 1.0
            
            
#         # sex
#         sex = current_wav_dict["sex"]
#         sex_feature = np.zeros(2, dtype=np.float32)
#         if compare_strings(sex, 'Female'):
#             sex_feature[0] = 1.0
#         elif compare_strings(sex, 'Male'):
#             sex_feature[1] = 1.0
            
#         # height and weight.
#         height = current_wav_dict["height"]
#         weight = current_wav_dict["weight"]
        
#         ## simple impute
#         if math.isnan(height) :
#             height = 110.846  #mean
#         if math.isnan(weight) :
#             weight = 23.767   #mean
            
#         height_weight = np.array([height, weight], dtype=np.float32)
        
        
#         # Extract pregnancy
#         preg_feature = np.zeros(2, dtype=np.float32)
#         is_pregnant = current_wav_dict["pregnancy_status"]
#         if is_pregnant == True:
#             preg_feature[0] = 1.0
#         elif is_pregnant == False:
#             preg_feature[1] = 1.0

#         # Extract location
#         location = current_wav_dict["location"]
#         num_recording_locations = len(self.recording_locations)
#         loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
#         if location in self.recording_locations:
#             j = self.recording_locations.index(location)
#             loc_feature[j] = 1.0


        

        # label
        
        current_murmur = np.zeros(self.num_murmur_classes, dtype=np.float32)
        murmur = get_murmur(current_patient_data)
        if murmur in self.murmur_classes:
            j = self.murmur_classes.index(murmur)
            current_murmur[j] = 1
        else:
            current_murmur[j] = 1


        current_outcome = np.zeros(self.num_outcome_classes, dtype=np.float32)
        outcome = get_outcome(current_patient_data)
        if outcome in self.outcome_classes:
            j = self.outcome_classes.index(outcome)
            current_outcome[j] = 1
            


        
        return feature, current_murmur, current_outcome

In [15]:
current_recording = read_recording('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/84780_AV.wav')

In [31]:
wav_files = glob.glob(args.train_data_folder + '*.wav')

In [32]:
wav_files[0]

'/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/84780_AV.wav'

In [23]:
torch.tensor(current_recording, dtype=torch.float32)

tensor([ -499.,  -575., -1087.,  ..., -3010.,    57.,  2857.])

In [18]:
current_recording.reshape(1,80000).shape

(1, 80000)

In [9]:
# Find data files.
# if verbose >= 1:
#     print('Finding data files...')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
    
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

# Find the patient data files.
patient_files = find_patient_files(args.train_data_folder)
num_patient_files = len(patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')

# Create a folder for the model if it does not already exist.
os.makedirs(args.model_path, exist_ok=True)
os.makedirs(args.logdir, exist_ok=True)

open(args.logdir + 'score.txt', "a+")
score_file = open(args.logdir + 'score.txt', "a+")


model_folder = args.model_path
log_dir = args.logdir
logger = get_logger(log_dir + '/' + model_folder)

# Define the model
lcnn_model = LCNN("murmur")
lcnn_model = lcnn_model.cuda(device)

# Define the metrics
accuracy = Accuracy(num_classes=3, multiclass=True)
f1 = F1(num_classes=3,average='macro', multiclass=True)

# optimizer, loss
opt = torch.optim.Adam(lcnn_model.parameters(), lr=args.lr, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
criterion = torch.nn.CrossEntropyLoss()


#data loader
trainset = train_loader(args.train_data_folder, args.train_feature_folder )
validset = train_loader(args.valid_data_folder, args.valid_feature_folder )
point_sampler = RandomSampler(trainset)

dataloader_train = DataLoader(dataset=trainset,
                    batch_size=args.minibatchsize_train,
                           sampler =point_sampler,
                           num_workers=args.train_num_workers)
dataloader_valid = DataLoader(dataset=validset,
                    batch_size=args.minibatchsize_valid,
                           num_workers=args.dev_num_workers)

Available devices  1


In [9]:
a, b, c  = next(iter(dataloader_valid))

In [10]:
len(validset)

631

In [ ]:
x = torch()

In [10]:
# # Find data files.
# if verbose >= 1:
#     print('Finding data files...')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print ('Available devices ', torch.cuda.device_count())
    
# GPU_NUM = args.gpu # 원하는 GPU 번호 입력
# device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device) # change allocation of current GPU

# # Find the patient data files.
# patient_files = find_patient_files(args.train_data_folder)
# num_patient_files = len(patient_files)

# if num_patient_files==0:
#     raise Exception('No data was provided.')

# # Create a folder for the model if it does not already exist.
# os.makedirs(args.model_path, exist_ok=True)
# os.makedirs(args.logdir, exist_ok=True)

# open(args.logdir + 'score.txt', "a+")
# score_file = open(args.logdir + 'score.txt', "a+")


# model_folder = args.model_path
# log_dir = args.logdir
# logger = get_logger(log_dir + '/' + model_folder)

# # Define the model
# lcnn_model = LCNN("murmur")
# lcnn_model = lcnn_model.cuda(device)

# # Define the metrics
# accuracy = Accuracy(num_classes=3, multiclass=True)
# f1 = F1(num_classes=3,average='macro', multiclass=True)

# # optimizer, loss
# opt = torch.optim.Adam(lcnn_model.parameters(), lr=args.lr, betas=(0.9, 0.999))
# scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
# criterion = torch.nn.CrossEntropyLoss()


# #data loader
# trainset = train_loader(args.train_data_folder)
# validset = train_loader(args.valid_data_folder)
# point_sampler = RandomSampler(trainset)

# dataloader_train = DataLoader(dataset=trainset,
#                     batch_size=args.minibatchsize_train,
#                            sampler =point_sampler,
#                            num_workers=args.train_num_workers)
# dataloader_valid = DataLoader(dataset=validset,
#                     batch_size=args.minibatchsize_valid,
#                            num_workers=args.dev_num_workers)

# Train the model.
# if verbose >= 1:
#     print('Training model...')
    
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']
if args.mode=='murmur':
    classes =  murmur_classes 
elif args.mode =='outcome':
    classes = outcome_classes


    
all_file_train = len(trainset)
dataloader_train_len = len(dataloader_train)
output_preds_labels_train = torch.empty((all_file_train,))
output_preds_probs_train = torch.empty((all_file_train,3))
output_gt_labels_train = torch.empty((all_file_train,))

for iter_ in range(args.end_iter):
    start_time = time.time()
    running_loss = 0.0
    idx = 0
    lcnn_model.train()
    for feature, current_murmur, current_outcome in tqdm(dataloader_train):
        feature = feature.cuda()
#         current_age_group = current_age_group.cuda()
#         sex_feature = sex_feature.cuda()
#         height_weight = height_weight.cuda()
#         preg_feature = preg_feature.cuda()
#         loc_feature = loc_feature.cuda()
        current_murmur = current_murmur.cuda()
        current_outcome = current_outcome.cuda()




        output_train = lcnn_model(feature)
        loss = criterion(output_train, current_murmur)



        output_probs_train = F.log_softmax(output_train, dim=1)

        _, output_label_argmax = torch.max(output_probs_train, dim=1)


        output_label_argmax = output_label_argmax.data.detach().cpu()

        output_preds_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train]= output_label_argmax  #murmur 예측 레이블 (0,1,2)

        output_preds_probs_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train,:] = output_probs_train
        
        

        _, murmur_label_argmax = torch.max(current_murmur.data, dim=1)  #murmur groud truth (0,1,2)

        output_gt_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train] = murmur_label_argmax.data.detach().cpu()



        idx +=1              

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_loss += loss.item()

    train_f1 = f1(preds = torch.tensor(output_preds_labels_train, dtype=torch.int32), target = torch.tensor(output_gt_labels_train, dtype=torch.int32))
    train_acc = accuracy(preds = torch.tensor(output_preds_labels_train, dtype=torch.int32), target = torch.tensor(output_gt_labels_train, dtype=torch.int32))
    

    scheduler.step()
    print("lr: ", opt.param_groups[0]['lr'])
    logger.info("Iteration:{0}, train loss = {1:.6f} ,train F1 = {2:.6f} ,train ACC = {3:.6f} ".format(iter_, 
                 running_loss/dataloader_train_len,train_f1,train_acc))

    # eval
    all_file_valid = len(validset)
    dataloader_valid_len = len(dataloader_valid)
    lcnn_model.eval()
    
    with torch.no_grad():
        running_loss_dev = 0.0
        index = 0
        output_preds_labels_dev = torch.empty((all_file_valid,))
        output_preds_labels_onehot_dev = torch.zeros((all_file_valid,3))
        output_preds_probs_dev = torch.empty((all_file_valid,3))
        output_gt_labels_dev= torch.empty((all_file_valid,))
        output_gt_labels_onehot_dev = torch.empty((all_file_valid,3))
        for feature,  current_murmur, current_outcome in tqdm(dataloader_valid):
            feature_dev = feature.cuda()
#             current_age_group_dev = current_age_group.cuda()
#             sex_feature_dev = sex_feature.cuda()
#             height_weight_dev = height_weight.cuda()
#             preg_feature_dev = preg_feature.cuda()
#             loc_feature_dev = loc_feature.cuda()
            current_murmur_dev = current_murmur.cuda()
            current_outcome_dev = current_outcome.cuda()


            output_dev = lcnn_model(feature_dev)
            loss_dev = criterion(output_dev, current_murmur_dev)                   
            running_loss_dev += loss_dev.item()




            output_probs_dev = F.log_softmax(output_dev, dim=1)

            _, output_label_argmax_dev = torch.max(output_probs_dev, dim=1)
            
            output_preds_labels_onehot_dev[index,output_label_argmax_dev]=1


            output_label_argmax_dev = output_label_argmax_dev.data.detach().cpu()

            output_preds_labels_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid]= output_label_argmax_dev  #murmur 예측 레이블 (0,1,2)

            output_preds_probs_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid,:] = output_probs_dev

            _, murmur_label_argmax_dev = torch.max(current_murmur_dev.data, dim=1)  #murmur groud truth (0,1,2)

            output_gt_labels_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid] = murmur_label_argmax_dev
            
            output_gt_labels_onehot_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid] = current_murmur_dev

            index = index+1   
            
        
        
        outputs = output_preds_labels_onehot_dev.data.detach().cpu().numpy()
        labels = output_gt_labels_onehot_dev.data.detach().cpu().numpy()


        dev_f1 = f1(preds = torch.tensor(output_preds_labels_dev, dtype=torch.int32), target = torch.tensor(output_gt_labels_dev, dtype=torch.int32))
        dev_acc = accuracy(preds = torch.tensor(output_preds_labels_dev, dtype=torch.int32),target =  torch.tensor(output_gt_labels_dev, dtype=torch.int32))
        weighted_acc = compute_weighted_accuracy(labels, outputs, classes)

        



    logger.info("valid Epoch:%d, Train loss=%.4f, Valid loss=%.4f,Valid F1=%.4f,Valid ACC=%.4f, Valid weighted acc=%.4f" %( iter_,
            running_loss/dataloader_train_len, running_loss_dev/dataloader_valid_len, dev_f1, dev_acc,weighted_acc ))
    score_file.write("%d epoch, VALID LOSS %f, VALID F1 %2.2f%% VALID ACC %2.2f%% VALID WEIGHTED ACC%2.2f%%\n"%(iter_, 
                            running_loss_dev/dataloader_valid_len, dev_f1, dev_acc,weighted_acc ))
    score_file.flush()

    torch.save(lcnn_model.state_dict(), os.path.join(model_folder, "{}_{}.model".format(args.model_name, iter_)))

    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
    logger.info("*" * 50)


# if verbose >= 1:
#     print('Done.')

100%|███████████████████████████████████████████| 80/80 [00:17<00:00,  4.66it/s]

lr:  0.00091
08-08 01:51 [MainProcess, 13692] [INFO ]  Iteration:0, train loss = 0.611032 ,train F1 = 0.417418 ,train ACC = 0.791074 



100%|████████████████████████████████████████| 631/631 [00:01<00:00, 398.10it/s]

08-08 01:51 [MainProcess, 13692] [INFO ]  valid Epoch:0, Train loss=0.6110, Valid loss=0.6327,Valid F1=0.4355,Valid ACC=0.7829, Valid weighted acc=0.5463
08-08 01:51 [MainProcess, 13692] [INFO ]  Time used for each epoch training: 18.78766417503357 seconds.
08-08 01:51 [MainProcess, 13692] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 80/80 [00:17<00:00,  4.70it/s]

...

08-08 02:15 [MainProcess, 13692] [INFO ]  valid Epoch:78, Train loss=0.1566, Valid loss=0.8356,Valid F1=0.4620,Valid ACC=0.8082, Valid weighted acc=0.5726
08-08 02:15 [MainProcess, 13692] [INFO ]  Time used for each epoch training: 18.688886165618896 seconds.
08-08 02:15 [MainProcess, 13692] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 80/80 [00:17<00:00,  4.68it/s]

lr:  5.288242786788324e-07
08-08 02:15 [MainProcess, 13692] [INFO ]  Iteration:79, train loss = 0.109650 ,train F1 = 0.819042 ,train ACC = 0.964060 



100%|████████████████████████████████████████| 631/631 [00:01<00:00, 393.09it/s]


08-08 02:15 [MainProcess, 13692] [INFO ]  valid Epoch:79, Train loss=0.1096, Valid loss=0.7656,Valid F1=0.4617,Valid ACC=0.7924, Valid weighted acc=0.5841
08-08 02:15 [MainProcess, 13692] [INFO ]  Time used for each epoch training: 18.73727774620056 seconds.
08-08 02:15 [MainProcess, 13692] [INFO ]  **************************************************


In [ ]:
def load_challenge_model(model_folder, verbose):
    murmur_train_model = torch.load(model_folder + '/' + args.model_name + '_63.model')
    outcome_train_model = torch.load(model_folder + '/' + args.outcome_model_name + '_63.model')
    return murmur_train_model, outcome_train_model

In [30]:
def load_challenge_model(model_folder, verbose):
    murmur_train_model = torch.load('/home/jh20/narin/physionet/hmd_LCNN' + '/' + 'hmd_LCNN_model_murmur_.model')
    outcome_train_model = torch.load('/home/jh20/narin/physionet/hmd_CNN_outcome' + '/' + 'hmd_CNN_outcome_model_25.model')
    return [murmur_train_model, outcome_train_model]

In [31]:
model = load_challenge_model(args.model_path, verbose=2)

In [15]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022/model")

from CNN_outcome import CNN as CNN_outcome
from LCNN_murmur import LCNN as LCNN_murmur

In [33]:
def run_challenge_model(model, data, recordings, verbose):
    murmur_model , outcome_model = model
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print ('Available devices ', torch.cuda.device_count())
        
    GPU_NUM = args.gpu # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU

    
    # Define the model
    murmur_classifier = LCNN_murmur("murmur")
    murmur_classifier = murmur_classifier.cuda(device)
    
    outcome_classifier = CNN_outcome()
    outcome_classifier = outcome_classifier.cuda(device)
    
    murmur_classifier.load_state_dict(murmur_model)
    outcome_classifier.load_state_dict(outcome_model)
    
#     wav_files = glob.glob(data_folder + '*.wav')
#     num_wav_files = len(wav_files)
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    recordings_list=[]
    for i in range(num_recordings):
        recording = recordings[i]
        length = 20*4000  
        if recording.shape[0] <= length:
            shortage = length - recording.shape[0]
            recording = np.pad(recording, (0, shortage), 'wrap')
        start_frame = np.int64(random.random()*(recording.shape[0]-length))
        recording = recording[start_frame:start_frame + length] 
        
        recordings_list.append(recording)
        

    features = get_features(data, recordings_list)
    
    predict_murmur_arr= np.zeros((num_recordings,3))
    predict_outcome_arr= np.zeros((num_recordings,2))
    
    for i in range(num_recordings):
        
        logmel = torch.tensor(features['mel'][i]).cuda()
        age = torch.tensor(features['age']).cuda()
        sex = torch.tensor(features['sex']).cuda()
        hw = torch.tensor(features['hw']).cuda()
        preg = torch.tensor(features['preg']).cuda()
        loc = torch.tensor(features['loc'][i]).cuda()
        wav2vec2_feature = torch.tensor(features['wav2vec2'][i]).cuda()
        
        logmel = logmel.unsqueeze(0)
        age = age.unsqueeze(0)
        sex = sex.unsqueeze(0)
        hw = hw.unsqueeze(0)
        preg = preg.unsqueeze(0)
        loc = loc.unsqueeze(0)
        wav2vec2_feature = wav2vec2_feature.unsqueeze(1)
        
        predict_murmur = F.log_softmax(murmur_classifier(wav2vec2_feature), dim=1)
        predict_outcome = F.log_softmax(outcome_classifier(logmel,age,sex,hw,preg,loc), dim=1)
        
        predict_murmur_arr[i,:]= predict_murmur.data.detach().cpu().numpy()
        predict_outcome_arr[i,:] = predict_outcome.data.detach().cpu().numpy()

    
    
    
   
    # Get classifier probabilities.
    idx1 = predict_murmur_arr.argmax(axis=0)[0]
    murmur_probabilities = predict_murmur_arr[idx1,] 
    idx2 = predict_outcome_arr.argmax(axis=0)[0]
    outcome_probabilities = predict_outcome_arr[idx2,]


    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

In [34]:
run_challenge_model(model, patient_data, recordings_list, verbose=2)

(['Present', 'Unknown', 'Absent', 'Abnormal', 'Normal'],
 array([0, 0, 1, 1, 0]),
 array([-2.87280416, -3.26627851, -0.099476  , -0.41523492, -1.07935441]))

In [35]:
import numpy as np, os, sys
# from helper_code import *
# from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [36]:
run_model(args.model_path, args.valid_data_folder, args.output_folder, allow_failures=True, verbose=2)

Loading Challenge model...
Running model on Challenge data...
    1/191...
    2/191...
    3/191...
    4/191...
    5/191...
    6/191...
    7/191...
    8/191...
    9/191...
    10/191...
    11/191...
    12/191...
    13/191...
    14/191...
    15/191...
    16/191...
    17/191...
    18/191...
    19/191...
    20/191...
    21/191...
    22/191...
    23/191...
    24/191...
    25/191...
    26/191...
    27/191...
    28/191...
    29/191...
    30/191...
    31/191...
    32/191...
    33/191...
    34/191...
    35/191...
    36/191...
    37/191...
    38/191...
    39/191...
    40/191...
    41/191...
    42/191...
    43/191...
    44/191...
    45/191...
    46/191...
    47/191...
    48/191...
    49/191...
    50/191...
    51/191...
    52/191...
    53/191...
    54/191...
    55/191...
    56/191...
    57/191...
    58/191...
    59/191...
    60/191...
    61/191...
    62/191...
    63/191...
    64/191...
    65/191...
    66/191...
    67/191...
    68/19

In [24]:
import librosa
import math
import random

def get_features(data, recordings):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    
    
    feature_dict={}
    
    feature_dict['mel']=[]
    for i in range(num_recordings):
        #log mel feature
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = recordings[i].astype(np.float32),
                                                         sr= 4000,
                                                             n_mels=128,
                                                             n_fft=400, 
                                                             hop_length=128, 
                                                             win_length=400))
        feature_dict['mel'].append(log_mel_feature)
    
    recording_info_lst = data.split('\n')[1:num_recordings+1]
    feature_dict['wav2vec2'] = []
    for i in range(num_recordings):
        pk_feature_fnm = recording_info_lst[i].split(' ')[2].replace('wav','pickle')
        with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/'+ pk_feature_fnm ,'rb') as fr:
            feature = pickle.load(fr)
        feature_dict['wav2vec2'].append(feature)
    

    # age
    current_patient_age = get_age(data)
    current_age_group = np.zeros(6, dtype=np.float32)
    if current_patient_age in age_classes:
        j = age_classes.index(current_patient_age)
        current_age_group[j] = 1.0
    else :
        current_age_group[5] = 1.0

    feature_dict['age']=current_age_group



    # sex
    sex = get_sex(data)
    sex_feature = np.zeros(2, dtype=np.float32)
    if compare_strings(sex, 'Female'):
        sex_feature[0] = 1.0
    elif compare_strings(sex, 'Male'):
        sex_feature[1] = 1.0

    feature_dict['sex']=sex_feature

    # height and weight.
    height = get_height(data)
    weight = get_weight(data)

    ## simple impute
    if math.isnan(height) :
        height = 110.846  #mean
    if math.isnan(weight) :
        weight = 23.767   #mean

    height_weight = np.array([height, weight], dtype=np.float32)


    feature_dict['hw']=height_weight

    # Extract pregnancy
    preg_feature = np.zeros(2, dtype=np.float32)
    is_pregnant = get_pregnancy_status(data)
    if is_pregnant == True:
        preg_feature[0] = 1.0
    elif is_pregnant == False:
        preg_feature[1] = 1.0

    feature_dict['preg']=preg_feature

    # Extract location
    feature_dict['loc']=[]
    locations = get_locations(data)
    for j in range(num_recordings):
        
        num_recording_locations = len(recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if locations[j] in recording_locations:
            idx = recording_locations.index(locations[j])
            loc_feature[idx] = 1.0
            
        feature_dict['loc'].append(loc_feature)




    # label

    current_murmur = np.zeros(num_murmur_classes, dtype=np.float32)
    murmur = get_murmur(data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    
    feature_dict['murmur']=current_murmur
    
    current_outcome = np.zeros(num_outcome_classes, dtype=np.float32)
    outcome = get_outcome(data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1

    feature_dict['outcome']=current_outcome
    
    


    return feature_dict

In [37]:
def evaluate_model(label_folder, output_folder):
    # Define murmur and outcome classes.
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']

    # Load and parse label and model output files.
    label_files, output_files = find_challenge_files(label_folder, output_folder)
    murmur_labels = load_murmurs(label_files, murmur_classes)
    murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
    outcome_labels = load_outcomes(label_files, outcome_classes)
    outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)

    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')

    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
        murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
        outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)

    # Return the results.
    return murmur_scores, outcome_scores

In [39]:
murmur_scores, outcome_scores = evaluate_model(args.valid_data_folder, args.output_folder)

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string


print(output_string)

with open('./output_score/score,txt', 'wt') as f:
    f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.495,0.361,0.281,0.728,0.375,25689.450

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.544,0.569,0.520,0.602,0.863,11768.189

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.624,0.327,0.535
AUPRC,0.289,0.058,0.735
F-measure,0.000,0.000,0.842
Accuracy,0.000,0.000,1.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.544,0.544
AUPRC,0.534,0.603
F-measure,0.719,0.321
Accuracy,0.990,0.194



In [ ]:
get_features(data, recordings)

In [18]:
patient_files = find_patient_files(args.valid_data_folder)
num_patient_files = len(patient_files)
patient_data = load_patient_data(patient_files[1])
recordings = load_recordings(args.valid_data_folder, patient_data)

import random
num_recordings = len(recordings)
recordings_list=[]
for i in range(num_recordings):
    recording = recordings[i]
    length = 20*4000  
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 
    
    recordings_list.append(recording)

In [19]:
recordings_list

[array([-2016,   710,   602, ...,   428,    89,  -253], dtype=int16),
 array([-5401, 19220,  9286, ..., -2441, -3624, -4292], dtype=int16)]

In [23]:
get_features(patient_data, recordings_list)['wav2vec2'][0].shape

torch.Size([1, 249, 768])

In [38]:
# Find Challenge files.
def find_challenge_files(label_folder, output_folder):
    label_files = list()
    output_files = list()
    for label_file in sorted(os.listdir(label_folder)):
        label_file_path = os.path.join(label_folder, label_file) # Full path for label file
        if os.path.isfile(label_file_path) and label_file.lower().endswith('.txt') and not label_file.lower().startswith('.'):
            root, ext = os.path.splitext(label_file)
            output_file = root + '.csv'
            output_file_path = os.path.join(output_folder, output_file) # Full path for corresponding output file
            if os.path.isfile(output_file_path):
                label_files.append(label_file_path)
                output_files.append(output_file_path)
            else:
                raise IOError('Output file {} not found for label file {}.'.format(output_file, label_file))

    if label_files and output_files:
        return label_files, output_files
    else:
        raise IOError('No label or output files found.')

# Load murmurs from label files.
def load_murmurs(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_murmur(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outcomes from label files.
def load_outcomes(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_outcome(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outputs from output files.
def load_classifier_outputs(output_files, classes):
    # The outputs should have the following form:
    #
    # #Record ID
    # class_1, class_2, class_3
    #       0,       1,       1
    #    0.12,    0.34,    0.56
    #
    num_patients = len(output_files)
    num_classes = len(classes)

    # Use one-hot encoding for the outputs.
    binary_outputs = np.zeros((num_patients, num_classes), dtype=np.bool_)
    scalar_outputs = np.zeros((num_patients, num_classes), dtype=np.float64)

    # Iterate over the patients.
    for i in range(num_patients):
        patient_id, patient_classes, patient_binary_outputs, patient_scalar_outputs = load_challenge_outputs(output_files[i])

        # Allow for unordered or reordered classes.
        for j, x in enumerate(classes):
            for k, y in enumerate(patient_classes):
                if compare_strings(x, y):
                    binary_outputs[i, j] = patient_binary_outputs[k]
                    scalar_outputs[i, j] = patient_scalar_outputs[k]

    return binary_outputs, scalar_outputs

# For each patient, set a specific class to positive if no class is positive or multiple classes are positive.
def enforce_positives(outputs, classes, positive_class):
    num_patients, num_classes = np.shape(outputs)
    j = classes.index(positive_class)

    for i in range(num_patients):
        if np.sum(outputs[i, :]) != 1:
            outputs[i, :] = 0
            outputs[i, j] = 1
    return outputs

# Compute macro AUROC and macro AUPRC.
def compute_auc(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    # Compute and summarize the confusion matrices for each class across at distinct output values.
    auroc = np.zeros(num_classes)
    auprc = np.zeros(num_classes)

    for k in range(num_classes):
        # We only need to compute TPs, FPs, FNs, and TNs at distinct output values.
        thresholds = np.unique(outputs[:, k])
        thresholds = np.append(thresholds, thresholds[-1]+1)
        thresholds = thresholds[::-1]
        num_thresholds = len(thresholds)

        # Initialize the TPs, FPs, FNs, and TNs.
        tp = np.zeros(num_thresholds)
        fp = np.zeros(num_thresholds)
        fn = np.zeros(num_thresholds)
        tn = np.zeros(num_thresholds)
        fn[0] = np.sum(labels[:, k] == 1)
        tn[0] = np.sum(labels[:, k] == 0)

        # Find the indices that result in sorted output values.
        idx = np.argsort(outputs[:, k])[::-1]

        # Compute the TPs, FPs, FNs, and TNs for class k across thresholds.
        i = 0
        for j in range(1, num_thresholds):
            # Initialize TPs, FPs, FNs, and TNs using values at previous threshold.
            tp[j] = tp[j-1]
            fp[j] = fp[j-1]
            fn[j] = fn[j-1]
            tn[j] = tn[j-1]

            # Update the TPs, FPs, FNs, and TNs at i-th output value.
            while i < num_patients and outputs[idx[i], k] >= thresholds[j]:
                if labels[idx[i], k]:
                    tp[j] += 1
                    fn[j] -= 1
                else:
                    fp[j] += 1
                    tn[j] -= 1
                i += 1

        # Summarize the TPs, FPs, FNs, and TNs for class k.
        tpr = np.zeros(num_thresholds)
        tnr = np.zeros(num_thresholds)
        ppv = np.zeros(num_thresholds)
        for j in range(num_thresholds):
            if tp[j] + fn[j]:
                tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
            else:
                tpr[j] = float('nan')
            if fp[j] + tn[j]:
                tnr[j] = float(tn[j]) / float(fp[j] + tn[j])
            else:
                tnr[j] = float('nan')
            if tp[j] + fp[j]:
                ppv[j] = float(tp[j]) / float(tp[j] + fp[j])
            else:
                ppv[j] = float('nan')

        # Compute AUROC as the area under a piecewise linear function with TPR/
        # sensitivity (x-axis) and TNR/specificity (y-axis) and AUPRC as the area
        # under a piecewise constant with TPR/recall (x-axis) and PPV/precision
        # (y-axis) for class k.
        for j in range(num_thresholds-1):
            auroc[k] += 0.5 * (tpr[j+1] - tpr[j]) * (tnr[j+1] + tnr[j])
            auprc[k] += (tpr[j+1] - tpr[j]) * ppv[j+1]

    # Compute macro AUROC and macro AUPRC across classes.
    if np.any(np.isfinite(auroc)):
        macro_auroc = np.nanmean(auroc)
    else:
        macro_auroc = float('nan')
    if np.any(np.isfinite(auprc)):
        macro_auprc = np.nanmean(auprc)
    else:
        macro_auprc = float('nan')

    return macro_auroc, macro_auprc, auroc, auprc

# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

# Define total cost for algorithmic prescreening of m patients.
def cost_algorithm(m):
    return 10*m

# Define total cost for expert screening of m patients out of a total of n total patients.
def cost_expert(m, n):
    return (25 + 397*(m/n) -1718*(m/n)**2 + 11296*(m/n)**4) * n

# Define total cost for treatment of m patients.
def cost_treatment(m):
    return 10000*m

# Define total cost for missed/late treatement of m patients.
def cost_error(m):
    return 50000*m

# Compute Challenge cost metric.
def compute_cost(labels, outputs, label_classes, output_classes):
    # Define positive and negative classes for referral and treatment.
    positive_classes = ['Present', 'Unknown', 'Abnormal']
    negative_classes = ['Absent', 'Normal']

    # Compute confusion matrix.
    A = compute_confusion_matrix(labels, outputs)

    # Identify positive and negative classes for referral.
    idx_label_positive = [i for i, x in enumerate(label_classes) if x in positive_classes]
    idx_label_negative = [i for i, x in enumerate(label_classes) if x in negative_classes]
    idx_output_positive = [i for i, x in enumerate(output_classes) if x in positive_classes]
    idx_output_negative = [i for i, x in enumerate(output_classes) if x in negative_classes]

    # Identify true positives, false positives, false negatives, and true negatives.
    tp = np.sum(A[np.ix_(idx_output_positive, idx_label_positive)])
    fp = np.sum(A[np.ix_(idx_output_positive, idx_label_negative)])
    fn = np.sum(A[np.ix_(idx_output_negative, idx_label_positive)])
    tn = np.sum(A[np.ix_(idx_output_negative, idx_label_negative)])
    total_patients = tp + fp + fn + tn

    # Compute total cost for all patients.
    total_cost = cost_algorithm(total_patients) \
        + cost_expert(tp + fp, total_patients) \
        + cost_treatment(tp) \
        + cost_error(fn)

    # Compute mean cost per patient.
    if total_patients > 0:
        mean_cost = total_cost / total_patients
    else:
        mean_cost = float('nan')

    return mean_cost


